# Inspection of DC2 Run 2.2i DR6 Object Table with Dask
### Michael Wood-Vasey (@wmwv)
### Last Verified to Run: 2020-07-08 by MWV

Inspect the Run 2.2i DR6 Object Table
Using a Dask Cluster on one NERSC node as the backend.

#### Run 2.2i DR6a as of 2020-06-04 includes  
  * 78 tracts
  * 52 million objects  
  * 34 million objects with i-band SNR > 5

2. Requires:
```
dask
dask.distributed
holoviews
datashader
bokeh
pyarrow >= 0.13.1
```

These were used with `desc-python-bleed` kernel

3. This was run using the `desc-python-bleed` kernel

We directly use the DPDD Parquet files.

## Import Needed Modules

In [ ]:
import math
import os

import numpy as np
from numpy.lib import scimath as SM

import astropy.units as u
import pandas as pd

In [ ]:
import colorcet

import dask
from dask.distributed import Client

import dask.dataframe as dd
import datashader as ds
import holoviews as hv
from holoviews.operation.datashader import datashade, shade, dynspread, rasterize
from holoviews.plotting.util import process_cmap

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
from matplotlib.patches import Polygon

import seaborn as sns

In [ ]:
hv.extension('bokeh')

In [ ]:
cmap = 'viridis'

## Start our Dask Cluster


For simple testing and illustration of how to use dask, holoview, and datashader here you can run locally on just one tract.
To run on the full set of DR6, you'll need to set up a node to support Dask distributed.  Basically you just need a machine that can hold the data in memory.

### Start a local Dask Cluster

In [ ]:
LOCAL_DASK = True

In [ ]:
if LOCAL_DASK:
    client = Client()

### Start a Dask Cluster on an Interactive Nodes

So instead, in a separate Terminal on Cori, ask for a pair of Node from the `interactive` queue.  This generally completes in seconds.  We ask for 2 Nodes because we'd like the full ~256 GB of availablel memory to store the data and the intermediate copies that often get made in some of the plots below:

Move to somewhere SCRATCH directory to simplify file locking
```
cd $SCRATCH/
```

```
salloc -N 2 -C haswell --qos=interactive -t 04:00:00
```

And then once on the first Node, where you'll get put after the `salloc` complets, load the right Python environment:
```
python /global/common/software/lsst/common/miniconda/start-kernel-cli.py desc-python-bleed
```

And then start up the Dask Cluster
```
NUM_WORKERS=16
SCHEDULER_FILE=${SCRATCH}/scheduler.json
dask-scheduler --scheduler-file ${SCHEDULER_FILE} &
dask-worker --nprocs ${NUM_WORKERS} --scheduler-file ${SCHEDULER_FILE} &
```

Then exit the environment and go to the second Node.

```
NUM_WORKERS=16
SCHEDULER_FILE=${SCRATCH}/scheduler.json
dask-worker --nprocs ${NUM_WORKERS} --scheduler-file ${SCHEDULER_FILE} &
```

The nodes will be printed out when the `salloc` launches.  And if you forget, you can look them up under the `SLURM_NODELIST` environment variable.

We connect to this Dask cluster through a shared agreement on where the `SCHEDULER_FILE` is.

We then configure the dashboard URL to use the JupyterHub proxy service.
We here set the formatting string template to the correct value.
Once we actually connect the client, then client can then tell us the full link.

In [ ]:
if not LOCAL_DASK:
    scheduler_file = os.path.join(os.environ["SCRATCH"], "scheduler.json")
    dask.config.config["distributed"]["dashboard"]["link"] = "{JUPYTERHUB_SERVICE_PREFIX}proxy/{host}:{port}/status"
    client = Client(scheduler_file=scheduler_file)

In [ ]:
client

## Define Catalog and Subsampling

In [ ]:
desc_data_dir = f"/global/cfs/cdirs/lsst/shared/DC2-prod/"
desc_data_dir = f"/Users/wmwv/tmp/DC2"

In [ ]:
data_release = "dr6a"

run_data_dir = f"Run2.2i/dpdd/Run2.2i-{data_release}/dc2_object_run2.2i_{data_release}"
datafile = os.path.join(desc_data_dir, run_data_dir)

In [ ]:
restrict_to_tracts = None
# restrict_to_tracts = [3640]

## Load Data

In [ ]:
filters = ('u', 'g', 'r', 'i', 'z', 'y')

In [ ]:
columns = ['ra', 'dec']
columns += [f'mag_{f}' for f in filters]
columns += [f'magerr_{f}' for f in filters]
columns += [f'mag_{f}_cModel' for f in filters]
columns += [f'magerr_{f}_cModel' for f in filters]
columns += [f'I_flag']
columns += [f'I_flag_{f}' for f in filters]
columns += [f'Ixx_{f}' for f in filters]
columns += [f'Ixy_{f}' for f in filters]
columns += [f'Iyy_{f}' for f in filters]
columns += [f'psf_fwhm_{f}' for f in filters]
columns += ['good', 'extendedness', 'blendedness']

In [ ]:
# Select good detections:
#  1. Marked as 'good' in catalog flags.
#  2. SNR in given band > threshold
#  3. In defined simulation range
snr_threshold = 5
snr_filter = 'i'

# We want to do a SNR cut, but magerr is the thing already calculated
# So we'll redefine our SNR in terms of magerr
magerr_cut = (2.5 / np.log(10)) / snr_threshold
snr_cut = f'magerr_{snr_filter} < {magerr_cut}'

In [ ]:
df = dd.read_parquet(datafile, columns=columns, engine='pyarrow')

In [ ]:
# Define color columns
df['u-g'] = df['mag_u'] - df['mag_g']
df['g-r'] = df['mag_g'] - df['mag_r']
df['r-i'] = df['mag_r'] - df['mag_i']
df['i-z'] = df['mag_i'] - df['mag_z']
df['z-y'] = df['mag_z'] - df['mag_y']

In [ ]:
# np.isfinite('blendedness')

good = df[df["good"] & (df[f"magerr_{snr_filter}"] < magerr_cut)]

In [ ]:
star = good[good['extendedness'] == 0]
galaxy = good[good['extendedness'] > 0]

In [ ]:
print(f'Total: {len(df)}, Good: {len(good)}, Stars: {len(star)}, Galaxies: {len(galaxy)}')

## Persist the data in Dask Cluster Worker memory

Dask actively purges data from memory when its no longer needed by the Dask Task Graph currently doing the computation.

That's not what we want here where we want to plot several quantities repeatedly.  So we explicitly tell Dask to persist this data frame.

In [ ]:
good = good.persist()

## Object Density in RA, Dec

DC2 Run 2.x WFD and DDF regions
https://docs.google.com/document/d/18nNVImxGioQ3tcLFMRr67G_jpOzCIOdar9bjqChueQg/view
https://github.com/LSSTDESC/DC2_visitList/blob/master/DC2visitGen/notebooks/DC2_Run2_regionCoords_WFD.ipynb

| Location          | RA (degrees) | Dec (degrees) | RA (degrees) | Dec (degrees) |
|:----------------- |:------------ |:------------- |:------------ |:------------- |
| Region            | WFD          | WFD           | DDF          | DDF           |
| Center            | 61.856114    | -35.79        | 53.125       | -28.100       |
| North-East Corner | 71.462228    | -27.25        | 53.764       | -27.533       |
| North-West Corner | 52.250000    | -27.25        | 52.486       | -27.533       |
| South-West Corner | 49.917517    | -44.33        | 52.479       | -28.667       |
| South-East Corner | 73.794710    | -44.33        | 53.771       | -28.667       |

(Note that the order of the rows above is different than in the DC2 papers.  The order of the rows above goes around the perimeter in order.)

In [ ]:
dc2_run2x_wfd = [[71.462228, -27.25], [52.250000, -27.25], [49.917517, -44.33], [73.794710, -44.33]]
dc2_run2x_ddf = [[53.764, -27.533], [52.486, -27.533], [52.479, -28.667], [53.771, -28.667]]

In [ ]:
dc2_run2x_wfd_df = pd.DataFrame({'ra': [coord[0] for coord in dc2_run2x_wfd] + [dc2_run2x_wfd[0][0]],
                                 'dec': [coord[1] for coord in dc2_run2x_wfd] + [dc2_run2x_wfd[0][1]]})
dc2_run2x_ddf_df = pd.DataFrame({'ra': [coord[0] for coord in dc2_run2x_ddf] + [dc2_run2x_ddf[0][0]],
                                 'dec': [coord[1] for coord in dc2_run2x_ddf] + [dc2_run2x_ddf[0][1]]})

In [ ]:
def plot_ra_dec_matplotlib(ra, dec, show_dc2_region=True, bins=100, cmin=10):
    """We're just doing this on a rectilinear grid.
    We should do a projection, of course, but that distortion is tolerable in this space."""
    fig = plt.figure(figsize=(8, 8))
    ax = plt.gca()
    ax.set_aspect(1)

    plt.hist2d(ra, dec, bins=bins, cmin=cmin)
    ax.invert_xaxis()  # Flip to East left
    plt.xlabel('RA [deg]')
    plt.ylabel('Dec [deg]')
    plt.colorbar(shrink=0.5, label='objects / bin')

    if show_dc2_region:
        # This region isn't quite a polygon.  The sides should be curved.
        wfd_region = Polygon(dc2_run2x_wfd, color='red', fill=False)
        ddf_region = Polygon(dc2_run2x_ddf, color='orange', fill=False)
        ax.add_patch(wfd_region)
        ax.add_patch(ddf_region)

        max_delta_ra = dc2_run2x_wfd[3][0] - dc2_run2x_wfd[2][0]
        delta_dec = dc2_run2x_wfd[1][1] - dc2_run2x_wfd[3][1]
        grow_buffer = 0.05
        ax.set_xlim(dc2_run2x_wfd[3][0] + max_delta_ra * grow_buffer,
                    dc2_run2x_wfd[2][0] - max_delta_ra * grow_buffer)
        ax.set_ylim(dc2_run2x_wfd[3][1] - delta_dec * grow_buffer,
                    dc2_run2x_wfd[1][1] + delta_dec * grow_buffer)

In [ ]:
def plot_ra_dec_hv(df, dc2_run2x_wfd_df=dc2_run2x_wfd_df, dc2_run2x_ddf_df=dc2_run2x_ddf_df,
                   show_dc2_region=True, cmap="bmy", bins=100, cmin=10):
    """We're just doing this on a rectilinear grid.
    We should do a projection, of course, but that distortion is tolerable in this space."""
    points_ra_dec = hv.Points(df, kdims=[hv.Dimension('ra', soft_range=(dc2_run2x_wfd[2][0], dc2_run2x_wfd[3][0])),
                                         hv.Dimension('dec', soft_range=(dc2_run2x_wfd[3][1], dc2_run2x_wfd[1][1]))])
    # We have to define the colormap here now, because the opts aren't passed through the datashade->Points.
    # See, e.g., https://github.com/holoviz/holoviews/issues/4125
    ra_dec = datashade(points_ra_dec, cmap=process_cmap(cmap, provider="colorcet"))
    ra_dec = ra_dec.opts(invert_xaxis=True)  # Flip to East left
    
    if show_dc2_region:
        # This region isn't quite a polygon.  The sides should be curved.
        wfd_region = hv.Path(dc2_run2x_wfd_df).opts(color='red')
        ddf_region = hv.Path(dc2_run2x_ddf_df).opts(color='orange')
        ra_dec = ra_dec * wfd_region * ddf_region
        
        max_delta_ra = dc2_run2x_wfd_df['ra'][3] - dc2_run2x_wfd_df['ra'][2]
        delta_dec = dc2_run2x_wfd_df['dec'][1] - dc2_run2x_wfd_df['dec'][3]
        grow_buffer = 0.05

        # Notice that these are specified in increasing RA left->right
        # We rely on the invert_xaxis True above to flip this in the display
        # It's important to get this right because these ranges are used for data selection
        # and then the range is flipped in the display.
        ra_dec.opts(xlim=(dc2_run2x_wfd[2][0] - max_delta_ra * grow_buffer,
                    dc2_run2x_wfd[3][0] + max_delta_ra * grow_buffer))
        ra_dec.opts(ylim=(dc2_run2x_wfd[3][1] - delta_dec * grow_buffer,
                    dc2_run2x_wfd[1][1] + delta_dec * grow_buffer))

    
    return ra_dec

In [ ]:
plot_ra_dec = plot_ra_dec_hv

In [ ]:
ra_dec = plot_ra_dec_hv(good)

In [ ]:
ra_dec

The overall object density distribution looks good.

Notes:
* We're missing the DDF region, which was specifically not included in this processing
* There are also a few patches that failed within the main region.
* There is an overall gradient N/S in object density, because we're plotting in rectilinear RA, Dec bins, which means that bins at the bottom in RA cover less area than those at the top.

See the input visit coverage map here:
https://github.com/LSSTDESC/ImageProcessingPipelines/issues/97#issuecomment-498303504


## Color-Color Diagrams and the Stellar Locus

In [ ]:
# We refer to a file over in `tutorials/assets' for the stellar locus
datafile_davenport = '../tutorials/assets/Davenport_2014_MNRAS_440_3430_table1.txt'

def get_stellar_locus_davenport(color1='gmr', color2='rmi',
                                datafile=datafile_davenport):
    color1 = color1.replace('-', 'm')
    color2 = color2.replace('-', 'm')

    data = pd.read_table(datafile, sep='\s+', header=1)
    return data[color1], data[color2]

    
def plot_stellar_locus(color1='gmr', color2='rmi',
                       color='blue', line_dash='dashed', line_width=2.5,
                       ax=None):

    color1_m = color1.replace('-', 'm')
    color2_m = color2.replace('-', 'm')

    model_color1, model_color2 = get_stellar_locus_davenport(color1_m, color2_m)
    model_df = pd.DataFrame({color1: model_color1, color2: model_color2})
    stellar_locus = hv.Path(model_df).opts(color='blue', line_dash=line_dash, line_width=line_width)
        
    return stellar_locus 

In [ ]:
def plot_color_color(df, color1, color2, 
                     range1=(-1, +2), range2=(-1, +2),
                     cmin=10, cmap='rainbow',
                     vmin=None, vmax=None):
    """Plot a color-color diagram.  Overlay stellar locus"""
    band1, band2 = color1[0], color1[-1]
    band3, band4 = color2[0], color2[-1]

    clean = df[np.isfinite(df[color1]) & np.isfinite(df[color2])]
    points_color1_color2 = hv.Points(
        clean,
        kdims=[
            hv.Dimension(color1, range=range1),
            hv.Dimension(color2, range=range2)]
    )

    color1_color2 = datashade(points_color1_color2, cmap=process_cmap(cmap, provider='colorcet'))
    return color1_color2
    try:
        stellar_locus = plot_stellar_locus(color1, color2)
        color1_color2 = color1_color2 * stellar_locus
    except KeyError as e:
        print(f"Couldn't plot Stellar Locus model for {color1}, {color2}")
        
    return color1_color2

In [ ]:
def plot_four_color_color(df, vmin=0, vmax=50000):
    layout = hv.Layout(
    plot_color_color(df, 'g-r', 'u-g') + \
    plot_color_color(df, 'g-r', 'r-i') + \
    plot_color_color(df, 'g-r', 'i-z') + \
    plot_color_color(df, 'g-r', 'z-y'))
    
    layout = layout.cols(2)
    return layout

In [ ]:
a = plot_color_color(df, 'g-r', 'u-g')

Note that the above panels will zoom in `g-r` together because Holovies knows that they share this data column.  They don't zoom "together" in the y-axes because those columns are not shared between the plots.

The plots each re-raster as you zoom in and out.

There is no brushing (selection) and linking.

In [ ]:
plot_four_color_color(star)

The discrete islands in the data for stellar color-color plot -- most visible in `r-i` vs. `g-r` at g-r ~= 1.2 mag -- are due to the finite set of stellar models used for simulating M dwarfs.

------
Let's plot the galaxies on the same color-color plots

Clearly one doesn't expect the galaxies to follow the stellar locus.  But including the stellar locus lines makes it easy to guide the eye between the stars-only and the galaxies-only plots.  

In [ ]:
plot_four_color_color(galaxy)

Questions for further study:
   1. Is there a better comparison sample for the stellar locus than the Davenport reference?
   2. Why is the stellar locus in the Davenport 0.1--0.2 mag redder for the reddest stars than the observed data.  Are there different extinction assumptions (this should be a low-extinction region).  Are there different bandpasses used?

## 1D Density Plots

To compare number densities, we have to calculate the area covered by each catalog.
We'll use Healpix through HealPy to pixelate the region and then count of the number of pixels with significant numbers of objects.

In [ ]:
def calculate_area(df, threshold=0.25, nside=1024, verbose=False):
    """Calculate the area covered by a catalog with 'ra', 'dec'
    
    Parameters:
    --
    cat: DataFrame, dict-like with 'ra', 'dec', keys
    threshold:  float
        Fraction of median value required to count a pixel.
    nside:  int
        Healpix NSIDE.  NSIDE=1024 is ~12 sq arcmin/pixel, NSIDE=4096 is 0.74 sq. arcmin/pixel
        Increasing nside will decrease calculated area as holes become better resolved 
        and relative Poisson fluctuations in number counts become more significant.
    verbose:  bool
        Print details on nside, number of significant pixels, and area/pixel.
        
    Returns:
    --
    area:  Astropy Quantity.
    """
    import healpy as hp

    indices = hp.ang2pix(nside, df['ra'], df['dec'], lonlat=True)
    idx, counts = np.unique(indices, return_counts=True)
    
    # Take the 25% of the median value of the non-zero counts/pixel
    threshold_counts = threshold * np.median(counts)

    if verbose:
        print(f'Median {np.median(counts)} objects/pixel')
        print(f'Only count pixels with more than {threshold_counts} objects')

    significant_pixels, = np.where(counts > threshold_counts)
    area_pixel = hp.nside2pixarea(nside, degrees=True) * u.deg**2

    if verbose:
        print(f'Pixel size ~ {hp.nside2resol(nside, arcmin=True) * u.arcmin:0.2g}')
        print(f'nside: {nside}, area/pixel: {area_pixel:0.4g}, num significant pixels: {len(significant_pixels)}')

    area = len(significant_pixels) * area_pixel

    if verbose:
        print(f'Total area: {area:0.7g}')
    
    return area

In [ ]:
area_dc2 = calculate_area(galaxy)
print(f'DC2 Run 2.2i area: {area_dc2:0.2f}')

In [ ]:
num_den_dc2 = len(galaxy) / area_dc2

# Change default expression to 1/arcmin**2
num_den_dc2 = num_den_dc2.to(1/u.arcmin**2)

In [ ]:
area_dc2

In [ ]:
area_dc2.unit

In [ ]:
def plot_mag_density_hist(df, filt, bins=None, density=False, area=None, color=None, **kwargs):
    mag_col = f'mag_{filt}'
    frequencies, edges = np.histogram(df[mag_col], bins=bins, density=density)
    if area is not None:
        frequencies = frequencies / area.value
    hist = hv.Histogram((edges, frequencies))
    hist.opts(xlabel=filt)
    
    if area is not None:
        ylabel = f"Objects/{area.unit}/bin"
    else:
        ylabel = "Objects/bin"
    hist.opts(ylabel=ylabel)

    if color:
        hist.opts(fill_color=None)
        hist.opts(line_color=color)
    
    return hist

def plot_mag_density_path(df, filt, object_type='', bins=None, density=False, area=None, **kwargs):
    mag_col = f'mag_{filt}'
    frequencies, edges = np.histogram(df[mag_col], bins=bins, density=density)
    if area is not None:
        frequencies = frequencies / area
    path = hv.Path(pd.DataFrame({mag_col: (edges[:-1]+edges[1:])/2, 'frequencies': frequencies}),
                  label=object_type)
#    if object_type is not None:
#        path.opts(label=object_type)

    path.opts(xlabel=mag_col)
    
    if area is not None:
        ylabel = f"Objects/{area.unit}/bin"
    else:
        ylabel = "Objects/bin"
    path.opts(ylabel=ylabel)
    
    return path 

plot_mag_density = plot_mag_density_hist

def plot_mag_densities(good, star, galaxy, filt,
                       area=None,
                       log=True, range=(16, 32), bins=None,
                       legend_position='top_left'):
    if bins is None:
        bins = np.linspace(*range, 100)
    
    densities = {'good': plot_mag_density(df, filt, object_type='good', bins=bins, area=area, color='green'),
                 'star': plot_mag_density(star, filt, object_type='star', bins=bins, area=area, color='blue'),
                 'galaxy': plot_mag_density(galaxy, filt, object_type='galaxy', bins=bins, area=area, color='red')}

    overlay = hv.NdOverlay(densities, kdims='Sample')
    overlay.opts(show_legend=True, legend_position=legend_position)
#    overlay.opts(logy=True)
    
    return overlay

In [ ]:
density_plots = [plot_mag_densities(good, star, galaxy, filt, area=area_dc2) for filt in filters]
density_plots[0].opts(legend_position='top_left')
layout = hv.Layout(density_plots)

In [ ]:
layout.cols(3)

The sharp cut in i-band is because that was the reference band for most detections.  The distributions in the other bands extend to 28th mag because many of the forced-photometry measurements are consistent with 0.

Notes:
1. I can't figure how to get Histograms to not have vertical lines.  It may not be possible because I think they're being drawn with Bokeh `quad`.
2. When I try to plot with `logy=True`, the plots become blank.
3. Yes, it really does alphabetize the samples.  I would like to learn how to force the order so I get can `galaxy` to appear more clearly on top of `good`.
4. If I instead use Path plot elements instead of Histogram, I can't figure out how to get labeled legends.

## Magnitude Error vs. Magnitude

The magnitude uncertainties come directly from the poisson estimates of the flux measurements.  By construction they will follow smooth curves.  We here confirm that they do.

In [ ]:
def plot_mag_magerr_matplotlib(df, band, ax=None, range=(16, 28), magerr_limit=0.25, vmin=100):
    # vmin=100 is reasonable for a ~50 million objects
    # If testing with one tract or a large subsampling factor, choose a smaller vmin.
    # Restrict to reasonable range
    mag_col, magerr_col = f'mag_{band}', f'magerr_{band}'
    good = df[df[magerr_col] < magerr_limit]

    ax.hexbin(good[mag_col], good[magerr_col], vmin=vmin)
    ax.set_xlabel(band)
    ax.set_ylabel(f'{band} err');
    ax.set_ylim(0, magerr_limit)

In [ ]:
def plot_mag_magerr(df, band, ax=None, range=(16, 28), magerr_limit=0.25, vmin=100,
                   cmap="rainbow", snr_magerr_threshold=magerr_cut):
    mag_col, magerr_col = f'mag_{band}', f'magerr_{band}'
    points_mag_magerr = hv.Points(df, kdims=[hv.Dimension(mag_col, range=(14, 28)),
                                             hv.Dimension(magerr_col, range=(0, snr_magerr_threshold))])
    return datashade(points_mag_magerr, cmap=process_cmap(cmap, provider='colorcet'))


In [ ]:
mag_magerr = hv.Layout([plot_mag_magerr(good, filt) for filt in filters])
mag_magerr.cols(3)

## Blendedness

Blendedness is a measure of how much the identified flux from an object is affected by overlapping from other objects.

See Bosch et al., 2018, Section 4.9.11.

In [ ]:
# print(f'{100 * len(w)/len(good_idx):0.1f}% of objects have finite blendedness measurements.')

Question for futher study:  What happened to yield non-finite blendedness measurements?

In [ ]:
blendedness = datashade(hv.Points(good, kdims=['mag_i', 'blendedness']), cmap=process_cmap("rainbow", provider="colorcet"))

In [ ]:
blendedness

### Extendedness
 
Extendedness is essentially star/galaxy separation based purely on morphology in the main detected reference band (which is `i` for most Objects).

Extendedness a binary property in the catalog, so it's either 0 or 1.

In [ ]:
extendedness = datashade(hv.Points(good, kdims=['mag_i', 'extendedness']), cmap=process_cmap("rainbow", provider="colorcet"))

In [ ]:
extendedness.opts(ylim=(-0.1, +1.1))

While the first plot above made extendedness look like a simple binary property, the truth is more complicated.

As galaxies get smaller in angular size and lower in signal-to-noise ratio, it becomes harder to clearly distinguish stars from galaxies.

Extendedness is based off of the difference between the point-source model and extended model brightness.  Specifically objects with `mag_psf - mag_cmodel > 0.164` mag are labeled with `extendedness=1` (i.e., galaxies).

See Bosch et al. 2018, Section 4.9.10 for details.

In [ ]:
plt.axvline(0.0164, 0.4, 1, color='red', linestyle='--',
           label=r'0.0164 $\Delta$mag cut')  # psf-cModel mag cut from Bosch et al. 2018.

plt.hist([df.loc[good_idx, 'mag_i'] - df.loc[good_idx, 'mag_i_cModel'],
         df.loc[star_idx, 'mag_i'] - df.loc[star_idx, 'mag_i_cModel'],
         df.loc[galaxy_idx, 'mag_i'] - df.loc[galaxy_idx, 'mag_i_cModel']],
         label=['Good', 'Stars', 'Galaxies'],
         bins=np.linspace(-0.1, 0.1, 201),
         histtype='step')

plt.legend()
plt.xlabel('mag_i[_psf] - mag_i_CModel');
plt.ylabel('objects / bin')

plt.text(0.25, 0.04, "STARS", transform=plt.gca().transAxes, fontdict={'fontsize': 24}, color='orange');
plt.text(0.65, 0.04, "GALAXIES", transform=plt.gca().transAxes, fontdict={'fontsize': 24}, color='orange')


In [ ]:
plt.hexbin(df.loc[good_idx, 'mag_i'], df.loc[good_idx, 'mag_i'] - df.loc[good_idx, 'mag_i_cModel'],
           extent=(14, 26, -0.75, +2.5),
           bins='log');
plt.xlabel('i')
plt.ylabel('mag_i[_psf] - mag_i_CModel');
plt.text(14.5, -0.5, "STARS", fontdict={'fontsize': 24}, color='orange')
plt.text(18, 2, "GALAXIES", fontdict={'fontsize': 24}, color='orange')
plt.colorbar(label='objects / bin');

plt.axhline(0.0164, 0.92, 1.0, color='red', linestyle='--')
plt.axhline(0.0164, 0, 0.1, color='red', linestyle='--',
            label=r'0.0164 $\Delta$mag cut');  # psf-cModel mag cut from Bosch et al. 2018.

We can zoom in a little to see how the fixed 0.0164 mag cut works at the low SNR limit.  Specifically at mag 24, we're starting to run out of stars and most things are galaxies.  But that's a population prior, it's not something visible using just morphology information.

You can see the effect of lower SNR measurements as the horizontal line at $\Delta$mag=0 puff up due to increased uncertainties.

In [ ]:
plt.hexbin(df.loc[good_idx, 'mag_i'], df.loc[good_idx, 'mag_i'] - df.loc[good_idx, 'mag_i_cModel'],
           extent=(22, 25.5, -0.1, +0.5),
           bins='log');
plt.xlabel('i')
plt.ylabel('mag_i[_psf] - mag_i_CModel');
plt.colorbar(label='objects / bin');

plt.axhline(0.0164, 0, 1, color='red', linestyle='--',
            label=r'0.0164 $\Delta$mag cut');  # psf-cModel mag cut from Bosch et al. 2018.

If we add in color information, 

In [ ]:
plt.hexbin(df.loc[good_idx, 'mag_g'] - df.loc[good_idx, 'mag_r'], df.loc[good_idx, 'mag_i'] - df.loc[good_idx, 'mag_i_cModel'],
           extent=(-2, +3, -0.5, +5),
           bins='log');
plt.xlabel('g-r')
plt.ylabel('mag_i[_psf] - mag_i_CModel');
# plt.text(14.5, 0.3, "STARS", fontdict={'fontsize': 24}, color='orange')
# plt.text(18, 2, "GALAXIES", fontdict={'fontsize': 24}, color='orange')
plt.colorbar(label='objects / bin');

In [ ]:
plt.hist([df.loc[galaxy_idx, 'mag_g'] - df.loc[galaxy_idx, 'mag_r'], df.loc[star_idx, 'mag_g'] - df.loc[star_idx, 'mag_r']],
        label=['galaxies', 'stars'], histtype='step',
        bins=np.linspace(-5, +5, 501));
plt.xlabel('g-r')
plt.ylabel('objects / bin')

In [ ]:
plt.hexbin(df.loc[star_idx, 'mag_g'] - df.loc[star_idx, 'mag_r'], df.loc[star_idx, 'mag_i'] - df.loc[star_idx, 'mag_i_cModel'],
           extent=(-2, +3, -0.5, +5),
           bins='log');
plt.xlabel('g-r')
plt.ylabel('mag_i[_psf] - mag_i_CModel');
# plt.text(14.5, 0.3, "STARS", fontdict={'fontsize': 24}, color='orange')
# plt.text(18, 2, "GALAXIES", fontdict={'fontsize': 24}, color='orange')
plt.colorbar(label='objects / bin');

## Shape Parameters

Ixx, Iyy, Ixy

In [ ]:
def plot_shape(df, filt, good_idx, star_idx, galaxy_idx, ax=None, legend=True):
    if not ax:
        ax = fig.gca()

    names = ['good', 'star', 'galaxy']
    colors = ['blue', 'orange', 'green']
    hist_kwargs = {'color': colors, 'log': True,
             'bins': np.logspace(-1, 1.5, 100),
             'range': (0, 50),
             'histtype': 'step'}
    for prefix, ls in (('Ixx', '-'), ('Iyy', '--'), ('Ixy', ':')):
        field = f'{prefix}_{filt}'
        labels = [f'{prefix} {name}' for name in names]
        ax.hist([df.loc[good_idx, field], df.loc[star_idx, field], df.loc[galaxy_idx, field]],
                label=labels,
                linestyle=ls,
                **hist_kwargs)

    ax.set_ylim(100, ax.get_ylim()[1])
    
    ax.set_xlabel(f'{filt}-band Moments: Ixx, Iyy, Ixy [pixels^2]')
    ax.set_ylabel('objects / bin')
    if legend:
        ax.legend()

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(12, 6))
legend = True
for ax, filt in zip(axes.flat, filters):
    plot_shape(df, filt, good_idx, star_idx, galaxy_idx, ax=ax, legend=legend)
    legend = False
plt.tight_layout()

The stars (orange) are concentrated at low values of the source moments.

Would be interesting to
1. Look by magnitude or SNR to undersatnd the longer tail.  Are these galaxies mis-classified as stars, or are these noise sources?
2. Distribution of ellipticity (see validate_drp to type this right)

In [ ]:
def ellipticity(I_xx, I_xy, I_yy):
    """Calculate ellipticity from second moments.

    Parameters
    ----------
    I_xx : float or numpy.array
    I_xy : float or numpy.array
    I_yy : float or numpy.array

    Returns
    -------
    e, e1, e2 : (float, float, float) or (numpy.array, numpy.array, numpy.array)
        Complex ellipticity, real component, imaginary component
        
    Copied from https://github.com/lsst/validate_drp/python/lsst/validate/drp/util.py
    """
    e = (I_xx - I_yy + 2j*I_xy) / (I_xx + I_yy + 2*SM.sqrt(I_xx*I_yy - I_xy*2))
    e1 = np.real(e)
    e2 = np.imag(e)
    return e, e1, e2

In [ ]:
def plot_ellipticity(df, filt, good_idx, star_idx, galaxie_idx, ax=None, legend=True):
    if not ax:
        ax = fig.gca()

    names = ['good', 'star', 'galaxy']
    colors = ['blue', 'orange', 'green']
    hist_kwargs = {'color': colors, 'log': True,
             'bins': np.logspace(-1, 1.5, 100),
             'range': (0, 5),
             'histtype': 'step'}
    for prefix, ls in (('e', '-'), ('e1', '--'), ('e2', ':')):
        field = f'{prefix}_{filt}'
        labels = [f'{prefix} {name}' for name in names]
        ax.hist([df.loc[good_idx, field], df.loc[star_idx, field], df.loc[galaxy_idx, field]],
                label=labels,
                linestyle=ls,
                **hist_kwargs)

    ax.set_xlim(0, 20)
    ax.set_ylim(10)
    
    ax.set_xlabel(f'{filt}-band ellipticity')
    ax.set_ylabel('objects / bin')
    if legend:
        ax.legend()

In [ ]:
for filt in filters:
    df[f'e_{filt}'], df[f'e1_{filt}'], df[f'e2_{filt}'] = \
    ellipticity(df[f'Ixx_{filt}'], df[f'Ixy_{filt}'], df[f'Iyy_{filt}'])

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(12, 6))
legend = True
for ax, filt in zip(axes.flat, filters):
    plot_ellipticity(df, filt, good_idx, star_idx, galaxy_idx, ax=ax, legend=legend)
    legend = False
plt.tight_layout()

### FWHM of the PSF
At the location of the catalog objects.

The Object Table stores the shape parameters of the PSF model as evaluated at the location of the object.

This is not the same as, but is certainly related to, the distribution of effective seeing in the individual images that made up the coadd.

In [ ]:
def plot_psf_fwhm(df, filters=filters,
                 colors=('purple', 'blue', 'green', 'orange', 'red', 'brown')):
    for filt, color in zip(filters, colors):
        psf_fwhm = np.array(df.loc[good_idx, f'psf_fwhm_{filt}'])
        w, = np.where(np.isfinite(psf_fwhm) & (psf_fwhm < 3))
        sns.distplot(psf_fwhm[w], label=filt, color=color)
    plt.xlabel('PSF FWHM [arcsec]')
    plt.ylabel('normalized object density')
    plt.legend()

In [ ]:
def plot_psf_fwhm(
    df,
    filters,
    colors=("purple", "blue", "green", "orange", "red", "brown"),
    max_psf_fwhm=1.5,
    plotname=None,
):
    bins = np.linspace(0, max_psf_fwhm, 201)
    for filt, color in zip(filters, colors):
        psf_fwhm = df[f"psf_fwhm_{filt}"].to_numpy()
        (w,) = np.where(np.isfinite(psf_fwhm) & (psf_fwhm < max_psf_fwhm))
        plt.hist(psf_fwhm[w], label=filt, color=color, histtype='step',
                bins=bins)
    plt.xlabel("PSF FWHM [arcsec]")
    plt.ylabel("Objects/bin")
    plt.legend()

    plt.tight_layout()
    if plotname is not None:
        plt.savefig(plotname)

In [ ]:
psf_fwhm_filters = [f"psf_fwhm_{filt}" for filt in filters]
plot_psf_fwhm(df.loc[good_idx, psf_fwhm_filters], filters)

This is the PSF model at the location of the objects on the coadd.